# Cluster bulk list of keywords with valueserp

In [2]:
api_key = "add key here"

In [ ]:
# create batch upload file



# include the fist 30 not just first page

In [1]:
# upload batch file to valueserp and get batch_id

batch_id = '903576B8'

In [4]:
# start the batch job

import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/start', params)

api_response = api_result.json()

print("Batch started.")


Batch started.


In [6]:
# check for job completed


import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id, params)

api_response = api_result.json()

print("Batch Name: ", api_response['batch']['name'])

api_response

Batch Name:  test


{'request_info': {'success': True},
 'batch': {'id': '903576B8',
  'created_at': '2023-02-11T03:57:53.219Z',
  'last_run': '2023-02-11T04:06:05.773Z',
  'name': 'test',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'idle',
  'searches_total_count': 47,
  'searches_page_count': 1,
  'credits_required': 47,
  'next_result_set_id': 2,
  'results_count': 1,
  'priority': 'normal',
  'destination_ids': [],
  'notification_email': 'alton@frontanalytics.com',
  'notification_as_json': True,
  'notification_as_jsonlines': False,
  'notification_as_csv': True,
  'notification_csv_fields': 'search.q,organic_results.position,organic_results.title,organic_results.link',
  'searches_type': 'web',
  'searches_per_type_count': {'web': 47},
  'searches_type_locked': True}}

In [11]:
# get the link for the download

import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/results/1/csv', params)

api_response = api_result.json()
api_response

{'request_info': {'success': True},
 'batch_id': '903576B8',
 'result': {'id': 1,
  'started_at': '2023-02-11T04:06:05.773Z',
  'ended_at': '2023-02-11T04:06:37.059Z',
  'expires_at': '2023-02-25T04:06:05.773Z',
  'results_page_count': 1,
  'searches_completed': 47,
  'searches_failed': 0,
  'searches_total': 47,
  'webhook_status': {'status': 'not_applicable', 'log': []},
  'destination_status': {},
  'download_links': {'pages': ['https://data.valueserp.com/results/11_FEBRUARY_2023/0406/Batch_Results_903576B8_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv'],
   'all_pages': 'https://data.valueserp.com/results/11_FEBRUARY_2023/0406/Batch_Results_903576B8_1_All_Pages_002c0139d00c9dd8e4d1b13c504517942bd20489_csv.zip'}}}

In [18]:
from io import BytesIO
from zipfile import ZipFile
import pandas
import requests

content = requests.get(url)
zf = ZipFile(BytesIO(content.content))

for item in zf.namelist():
    print("File in zip: "+  item)

# find the first matching csv file in the zip:
match = [s for s in zf.namelist() if ".csv" in s][0]
# the first line of the file contains a string - that line shall de     ignored, hence skiprows
df = pandas.read_csv(zf.open(match), low_memory=False)

File in zip: Batch_Results_903576B8_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv


In [25]:
# make dataframe from the serp lists
df = df[['result.organic_results.link', 'search.q', 'result.organic_results.position']]
df = df.rename(columns={'result.organic_results.link':'link', 'search.q':'query', 'result.organic_results.position':'position'})
df.head()

,link,query,position
0,https://www.komoot.com/guide/2145269/road-cycl...,bella villa bike rides,1
1,https://ridewithgps.com/trips/24810732,bella villa bike rides,2
2,https://www.bikemap.net/en/l/4101114/,bella villa bike rides,3
3,https://www.bellavistaar.gov/residents/recreat...,bella villa bike rides,4
4,https://m.facebook.com/people/Bella-Vista-Bike...,bella villa bike rides,5


In [30]:
# cluster the results

#common_urls = st.sidebar.slider("Set number of common urls to match", value=3, min_value=2, max_value=5)
common_urls = 3

def cluster_table( df, common_urls ):
    # -------------------------------- start cluster logic -------------------------------------------------------------
    df = df.sort_values(by="query", ascending=True)
    df = df.groupby('link')['query'].apply(','.join).reset_index()
    df["temp_count"] = (df["query"].str.count(",") + 1)
    df = df[df["temp_count"] >= common_urls]

    # keep only the longest version of each keyword group
    list1 = df['query']
    substrings = {w1 for w1 in list1 for w2 in list1 if w1 in w2 and w1 != w2}
    longest_str = set(list1) - substrings
    longest_str = list(longest_str)
    df = df[df['query'].isin(longest_str)]

    # drop any cluster groups under the minimum threshold
    df['temp_count'] = df['query'].map(df.groupby('query')['query'].count())
    df = df[df["temp_count"] >= common_urls]

    # make the cluster column & delete helper column
    df['serp_cluster'] = df['query']

    # explode the keyword list
    df['query'] = df['query'].str.split(',')
    df = df.explode('query')

    return df

In [31]:
cluster_table(df, common_urls)

,link,query,temp_count,serp_cluster
0,https://apps.cnt.org/bikechi/,bike lanes chicago,5,"bike lanes chicago,bike lanes in chicago,chica..."
0,https://apps.cnt.org/bikechi/,bike lanes in chicago,5,"bike lanes chicago,bike lanes in chicago,chica..."
0,https://apps.cnt.org/bikechi/,chicago bike lanes,5,"bike lanes chicago,bike lanes in chicago,chica..."
1,https://bikechicago.com/map/,bike lanes chicago,5,"bike lanes chicago,bike lanes in chicago,chica..."
1,https://bikechicago.com/map/,bike lanes in chicago,5,"bike lanes chicago,bike lanes in chicago,chica..."
1,https://bikechicago.com/map/,chicago bike lanes,5,"bike lanes chicago,bike lanes in chicago,chica..."
2,https://dpw.lacounty.gov/bike/map.cfm,bike lanes in los angeles,6,"bike lanes in los angeles,bike lanes los angel..."
2,https://dpw.lacounty.gov/bike/map.cfm,bike lanes los angeles,6,"bike lanes in los angeles,bike lanes los angel..."
2,https://dpw.lacounty.gov/bike/map.cfm,los angeles bike lanes,6,"bike lanes in los angeles,bike lanes los angel..."
3,https://la.streetsblog.org/2022/08/23/l-a-city...,bike lanes in los angeles,6,"bike lanes in los angeles,bike lanes los angel..."


In [ ]:
# save the cluster file


